Trying the QRF model first, then will go for modifications / further improvements 

In [1]:
from quantum_random_forest import QuantumRandomForest, set_multiprocessing
from split_function import SplitCriterion
from data_construction import data_preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics, datasets

2025-04-03 20:54:16.099892: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 20:54:16.367052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743713656.434644    1298 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743713656.448382    1298 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743713656.563857    1298 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
data = datasets.load_breast_cancer()
num_classes = 2
X, y = data.data, data.target

In [3]:
training_set, testing_set = data_preprocessing(X, y, 
                                               train_prop=0.75,        # Proportion of dataset allocated for training
                                               X_dim=6)                # Determine the required dimension of the dataset. None for default.

New datapoint dimension: 6


2025-04-03 20:54:24.984982: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
n_qubits = 6                                         # Number of qubits for the embedding
dt_type = 'qke'                                      # Do not touch
ensemble_var = None                                  # Do not touch
branch_var = ['eff_anz_pqc_arch', 
              'iqp_anz_pqc_arch', 
              'eff_anz_pqc_arch']                    # Type of Anzatz, or as a list for different down the tree - as given 
num_trees = 3                                        # Number of trees in ensemble 
split_num = 2                                        # Do not touch
pqc_sample_num = 2024                                # Number of circuit samples per kernel estimation
num_classes = num_classes                            # Number of classes in dataset
max_depth = 4                                        # Maximum depth of the tree
num_params_split = n_qubits*(n_qubits +1)            # Number of parameters in the embedding (different for different anzatz), list for different down the tree [2 * n_qubits ** 2 , n_qubits*(n_qubits +1), 2 * n_qubits ** 2]
num_rand_gen = 1                                     # Do not touch
num_rand_meas_q = n_qubits                           # Do not touch 
svm_num_train = 5                                    # L, Number of Landmarks
svm_c = 10                                           # C term in SVM optimisation, or list down the tree [100, 50, 20]
min_samples_split = svm_num_train                    # Minimum number of samples
embedding_type = ['as_params_all', 
                  'as_params_iqp', 
                  'as_params_all']                   # Type of embedding, or as a list - as given
criterion = SplitCriterion.init_info_gain('clas')    # Do not touch
device = 'cirq'                                      # Choose a device. Also possible to run on IBM

In [5]:
qrf = QuantumRandomForest(n_qubits, 'clas', num_trees, criterion, max_depth=max_depth, 
                          min_samples_split=min_samples_split, tree_split_num=split_num, num_rand_meas_q=num_rand_meas_q,
                          ensemble_var=ensemble_var, dt_type=dt_type, num_classes=num_classes, ensemble_vote_type='ave',
                          num_params_split=num_params_split, num_rand_gen=num_rand_gen, pqc_sample_num=pqc_sample_num,
                          embed=embedding_type, branch_var=branch_var, svm_num_train=svm_num_train, svm_c=svm_c, 
                          nystrom_approx=True, device=device)

In [6]:
cores = 3
set_multiprocessing(True, cores)                   # Set to False if you don't want parallel computation
qrf.train(training_set, 
          partition_sample_size=180)  


---Training sub-tree of depth:
---Training sub-tree of depth: 1  1 (180 instances)(180 instances)


---Training sub-tree of depth: 1 (180 instances)


100%|██████████| 25/25 [00:00<00:00, 37.39it/s]

100%|██████████| 900/900 [00:24<00:00, 36.90it/s]


Info gain: 0.5387
Accuracy for binary dataset: 0.9167

 94%|█████████▍| 846/900 [00:24<00:01, 46.29it/s]


Number of SV: [31 32]
----> Selected SVM info gain: 0.5387

---Training sub-tree of depth: 2 (57 instances)


...

Info gain: 0.0000
Accuracy for binary dataset: 0.0672
Number of SV: [16  8]
Increase SVM_C...


100%|██████████| 595/595 [00:09<00:00, 60.30it/s]


Info gain: 0.0000
Accuracy for binary dataset: 0.0672
Number of SV: [31  8]
Increase SVM_C...


100%|██████████| 595/595 [00:09<00:00, 62.16it/s]


Info gain: 0.0335
Accuracy for binary dataset: 0.0588
Number of SV: [9 8]
----> Selected SVM info gain: 0.0335

---Training sub-tree of depth: 4 (118 instances)

---Training sub-tree of depth: 4 (1 instances)

---Training sub-tree of depth: 3 (4 instances)


In [7]:
acc, preds_qrf = qrf.test(testing_set, 
                          ret_pred=True, 
                          parallel=False,            # Set to False if you don't want parallel computation. Needs to be False for calc_tree_corr to be True.
                          calc_tree_corr=True)  

100%|██████████| 143/143 [03:09<00:00,  1.33s/it]


In [8]:
# Classification report
print(f"Classification report for QRF:\n"
      f"{metrics.classification_report(testing_set.y, preds_qrf)}\n")

Classification report for QRF:
              precision    recall  f1-score   support

           0       0.86      0.83      0.84        52
           1       0.90      0.92      0.91        91

    accuracy                           0.89       143
   macro avg       0.88      0.88      0.88       143
weighted avg       0.89      0.89      0.89       143


